# Distance based time series classification in aeon

Distance based classifiers use a time series specific distance function to measure the
similarity between time series. Time series distance functions are
often called elastic distances, since they compensate for possible misalignment
between series by shifting or editing the series.

Dynamic time warping is the best known elastic distance measure. This image
demonstrates how a warping path is found between two series
<img src="./img/dtw.png" width="400" alt="A visualisation of dynamic time warping">

We have a range of elastic distance functions in the distances module. Please see the
 distances notebook for more information. Distance functions have been mostly used
 with a nearest neighbour (NN) classifier.

## Load data and list distance based classifiers

In [2]:
from sktime.datasets import load_basic_motions, load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_X_y=True)
X_test, y_test = load_italy_power_demand(split="test", return_X_y=True)
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_mv, y_train_mv = load_basic_motions(split="train", return_X_y=True)
X_test_mv, y_test_mv = load_basic_motions(split="test", return_X_y=True)

X_train_mv = X_train_mv[:20]
y_train_mv = y_train_mv[:20]
X_test_mv = X_test_mv[:20]
y_test_mv = y_test_mv[:20]

print(X_train_mv.shape, y_train_mv.shape, X_test_mv.shape, y_test_mv.shape)

(67, 1) (67,) (50, 1) (50,)
(20, 6) (20,) (20, 6) (20,)


## Distance based classifiers

We have a range of distance based

In [3]:
from sktime.registry import all_estimators

# search for all classifiers that can handle multivariate time series. This will
# give some UserWarnings if soft dependencies are not installed.
all_estimators(
    filter_tags={"classifier_type": "distance"}, estimator_types="classifier"
)

[('ElasticEnsemble',
  sktime.classification.distance_based._elastic_ensemble.ElasticEnsemble),
 ('MatrixProfileClassifier',
  sktime.classification.feature_based._matrix_profile_classifier.MatrixProfileClassifier),
 ('ShapeDTW', sktime.classification.distance_based._shape_dtw.ShapeDTW)]

In [ ]:
from sktime.classification.distance_based import (
    ElasticEnsemble,
    KNeighborsTimeSeriesClassifier,
    ShapeDTW,
)

shape = ShapeDTW()
ee = ElasticEnsemble()

## K-NN
k-NN is often called a lazy classifier, because there is little work done in
the fit operation. The fit operation simply stores the training data. When we want to
make a prediction for a new time series, k-NN measures the distance between the new
time series and all the series in the training data and records the class of the
closest k train series. The class labels of these nearest neighbours are used to make
 a prediction: if they are all the same label, then that is the prediction. If they
 differ, then some form of voting mechanism is required. For example, we may predict
 the most common class label amongst the nearest neighbours for the test instance.

KNeighborsTimeSeriesClassifier in aeon is configurable to use any of the distances
functions in the distance module, or it can be passed a bespoke callable. You can set
 the number of neighbours and the weights. Weights are used in the prediction
 process when neightbours differ in class values. By default all neighbours have an
 equal vote. There is an option to weight by distance, meaning closer neighbours have
  more weight in the vote.

In [ ]:
knn = KNeighborsTimeSeriesClassifier(distance="msm", n_neighbors=3, weights="distance")


## Elastic Ensemble


## References

